In [69]:
# Lectura de librerías
import pdfplumber
import pandas as pd
import os

In [70]:
# Ruta donde están los PDFs
carpeta_pdfs = rf'C:\Alejo\PreciosBolsaEnersinc\Informacion_20250716\UPME'
archivos_pdf = [f for f in os.listdir(carpeta_pdfs) if f.endswith('.pdf')]

# Lista para guardar todas las tablas
df_data = pd.DataFrame()

l_col=['Nombre del Proyecto','Descripción','Empresa','Capacidad(MW)','Tipo / Recurso','OEF','CLPE-02-2019'
       ,'Garantía','Punto de Conexión','Expansión / Condición','FPO','Informe_Auditoria','Estado','archivo_origen','pagina_origen']

for archivo in archivos_pdf:
    ruta_pdf = os.path.join(carpeta_pdfs, archivo)
    print(f"Procesando: {archivo}")

    with pdfplumber.open(ruta_pdf) as pdf:
        for i, pagina in enumerate(pdf.pages):
            try:
                tablas = pagina.extract_tables()
                for tabla in tablas:
                    if tabla and len(tabla) > 1:  # Verifica que no esté vacía
                        df = pd.DataFrame(tabla[1:], columns=tabla[0])  # Usa primera fila como encabezado
                        df['archivo_origen'] = archivo
                        df['pagina_origen'] = i + 1

                        # renombrar los nombres de las columnas de acuerdo con el número
                        df.columns = [f'col_{i+1}' for i in range(df.shape[1])]

                        # Detectar si la columna 1 tiene Nomvre de proyecto
                        if df['col_1'].astype(str).str.contains('Nombre del Proyecto').any():

                            for col in df.columns:
                                missing_ratio = df[col].isna().mean()
                                if missing_ratio > 0.8:
                                    df.drop(columns=col, inplace=True)

                            # Elimiar la fila que tenga Nombre del proyecto en la columna col_1
                            df = df[df['col_1'] != 'Nombre del Proyecto']

                            # Eliminar filas donde más del 50% de las columnas sean missing values o estén vacío
                            df = df[df.isnull().mean(axis=1) <= 0.2]

                            # Cambiar los nombre de las columnas por los de l_col
                            df.columns = l_col[:df.shape[1]]

                            # Elimiar filas donde la columna Nombre del proyecto y la columna Estado estén vacías
                            df = df[~(df['Nombre del Proyecto'].isna() | df['Nombre del Proyecto'].eq('')) & ~(df['Estado'].isna() | df['Estado'].eq(''))]


                            # Concatnenar las tablas
                            df_data=pd.concat([df_data,df],axis=0)

            except Exception as e:
                print(f"Error en página {i+1} de {archivo}: {e}")



Procesando: Informe_avance_proyectos_generacion_Abril_2024.pdf
Procesando: Informe_Avance_proyectos_Generacion_Diciembre2020.pdf
Procesando: Informe_Avance_proyectos_Generacion_Diciembre2021.pdf
Error en página 9 de Informe_Avance_proyectos_Generacion_Diciembre2021.pdf: Length mismatch: Expected axis has 17 elements, new values have 15 elements
Procesando: Informe_Avance_proyectos_Generacion_Diciembre_2022.pdf
Procesando: Informe_avance_proyectos_generacion_Diciembre_2023.pdf
Procesando: Informe_avance_proyectos_generacion_Diciembre_2024.pdf
Procesando: Informe_Avance_proyectos_Generacion_Julio2022.pdf
Procesando: Informe_Avance_proyectos_Generacion_Julio_2023.pdf
Procesando: Informe_avance_proyectos_generacion_Julio_2024.pdf
Procesando: Informe_Avance_proyectos_Generacion_Junio2021.pdf
Error en página 9 de Informe_Avance_proyectos_Generacion_Junio2021.pdf: Length mismatch: Expected axis has 16 elements, new values have 15 elements
Procesando: Informe_Avance_proyectos_Generacion_Marzo2

In [71]:
df_data.to_csv('Información proyectos.csv', index=False, encoding='utf-8-sig')

# df_data.to_csv('Información proyectos.csv')

In [ ]:
import re
import unicodedata


PALABRAS_COMUNES = {
    "proyecto", "parque", "solar", "fotovoltaico", "eólico", "hidroeléctrico", 
    "de", "del", "la", "el", "central", "planta", "generación", "energia","eolico", "autog", "agpe"
}

def normalizar_nombre(nombre):
    if not isinstance(nombre, str):
        return ""
    # Elimina tildes
    nombre = unicodedata.normalize('NFKD', nombre).encode('ascii', 'ignore').decode('utf-8')
    # Minúsculas
    nombre = nombre.lower()
    # Quita puntuación y símbolos
    nombre = re.sub(r'[^\w\s]', '', nombre)
    # Quita palabras comunes
    palabras = nombre.split()
    palabras_filtradas = [p for p in palabras if p not in PALABRAS_COMUNES]
    return " ".join(palabras_filtradas)

In [46]:
# from thefuzz import process
from fuzzywuzzy import fuzz, process

# --- Paso 2: función de fuzzy join ---
def fuzzy_left_join(df_izq, col_izq, df_der, col_der, umbral=85):
    # Limpieza de nombres
    df_izq["nombre_limpio"] = df_izq[col_izq].apply(normalizar_nombre)
    df_der["nombre_limpio"] = df_der[col_der].apply(normalizar_nombre)

    # Lista de referencia (derecha)
    nombres_ref = df_der["nombre_limpio"].tolist()

    # Buscar mejor match para cada fila del DF izquierdo
    matches = df_izq["nombre_limpio"].apply(
        lambda x: process.extractOne(x, nombres_ref,scorer=fuzz.ratio) if x else (None, 0)
    )

    # Crear columnas de match y score
    df_izq["match_nombre"] = matches.apply(lambda x: x[0])
    df_izq["similitud"] = matches.apply(lambda x: x[1])

    # Filtrar solo coincidencias aceptables
    df_match = df_izq[df_izq["similitud"] >= umbral].copy()

    # Hacer el join con los datos del DataFrame derecho
    df_final = pd.merge(
        df_match,
        df_der,
        left_on="match_nombre",
        right_on="nombre_limpio",
        suffixes=("_izq", "_der"),
        how="left"
    )

    return df_final

### Obtener información de las hojas

In [4]:
import pandas as pd

In [5]:
# Leer archivo de UPME
df_upme = pd.read_csv(rf'C:\Alejo\PreciosBolsaEnersinc\Informacion_20250716\Información_UPME.csv', header=0, encoding='utf-8-sig')
df_upme=df_upme[(df_upme['Tipo'].isin(['Eólico','Solar']))]

In [6]:
proyectos_unicos = df_upme['Nombre del Proyecto'].unique()
df_proyectos = pd.DataFrame()

for proyecto in proyectos_unicos:
    df_filtrado = df_upme[df_upme['Nombre del Proyecto'] == proyecto]
    df_filtrado = df_filtrado[df_filtrado['FPO'].notna() & (df_filtrado['FPO'] != '')]
    registro = {
        'Proyecto': proyecto,
        'FPO_Ini': df_filtrado['FPO'].min(),
        # 'FPO_max': df_filtrado['FPO'].max(),
        'CEN': df_filtrado['CEN'].iloc[0],
        'Despacho': df_filtrado['Despacho'].iloc[0],
        'Tipo': df_filtrado['Tipo'].iloc[0],
        'OEF': df_filtrado['OEF'].iloc[0]
    }
        
    df_proyectos = pd.concat([df_proyectos, pd.DataFrame([registro])], ignore_index=True)

    df_proyectos.fillna(0, inplace=True)


In [7]:
df_proyectos["nombre_normalizado"] = df_proyectos["Proyecto"].apply(normalizar_nombre)
df_proyectos

,Proyecto,FPO_Ini,CEN,Despacho,Tipo,OEF,nombre_normalizado
0,WINDPESHI,2022-03-31,200,DC,Eólico,0.78,windpeshi
1,TOLIMA NORTE,2025-12-31,50,DC,Solar,0.096,tolima norte
2,SOLAR EL PASO,2023-11-30,70,DC,Solar,0.24,paso
3,SHANGRI LA,2025-12-31,160,DC,Solar,0.54,shangri
4,SAN FELIPE,2020-12-31,90,DC,Solar,0,san felipe
5,PV BARZALOSA,2026-03-31,100,DC,Solar,0.21,pv barzalosa
6,PROYECTO PARQUE SOLAR VILLAVIEJA DE 200 MW,2026-11-15,200,DC,Solar,0.4,villavieja 200 mw
7,PLANTA SOLAR LA ORQUÍDEA,2026-11-12,200,DC,Solar,0.334,orquidea
8,PARUQE EÓLICO BETA,2026-11-01,280,DC,Eólico,0.2,paruqe beta
9,PARQUE SOLAR PUERTA DE ORO,2025-12-31,300,DC,Solar,0.6,puerta oro


In [8]:
import pandas as pd

# Ruta del archivo Excel
ruta_excel = r'C:\Alejo\PreciosBolsaEnersinc\Informacion_20250716\Seguimiento_radar_marzo_2024.xlsx'

# Leer todas las hojas del archivo
xls = pd.ExcelFile(ruta_excel)
hojas_filtradas = [hoja for hoja in xls.sheet_names if any(str(año) in hoja for año in range(2018, 2025))]

df_data_radar = pd.DataFrame()
# Recorrer cada hoja filtrada y leer su contenido
for hoja in hojas_filtradas:
    df_hoja = pd.read_excel(xls, sheet_name=hoja)
    df_hoja = df_hoja[['PROYECTO', 'FPO']]
    df_hoja = df_hoja.rename(columns={'FPO': 'FPO_Ini'})
    df_hoja = df_hoja[df_hoja['PROYECTO'].notna() & (df_hoja['PROYECTO'] != '')]
    df_hoja['origen']=hoja

    df_data_radar=pd.concat([df_data_radar,df_hoja],axis=0)
 


In [9]:
df_data_radar["nombre_normalizado"] = df_data_radar["PROYECTO"].apply(normalizar_nombre)
df_data_radar

,PROYECTO,FPO_Ini,origen,nombre_normalizado
0,Hidroituango,2018-11-01 00:00:00,marzo 2018,hidroituango
1,Ambeima,no tiene fecha probable de entrada en operación,marzo 2018,ambeima
2,Gecelca 3.2,2018-04-30 00:00:00,marzo 2018,gecelca 32
4,Porvenir II,2023,marzo 2018,porvenir ii
5,La Luna,2022-06-01 00:00:00,marzo 2018,luna
...,...,...,...,...
176,Chorreritas,NaN,marzo 2024,chorreritas
177,PSF Mangales,NaN,marzo 2024,psf mangales
178,El Roble 19.5,2025-12-31 00:00:00,marzo 2024,roble 195
179,La Cayena 19.9,2025-12-31 00:00:00,marzo 2024,cayena 199


In [10]:
from pydataxm import *                           #Se realiza la importación de las librerias necesarias para ejecutar                        
from pydataxm.pydataxm import ReadDB as apiXM 
import datetime as dt                            
import pandas as pd
from pathlib import Path
import os

In [11]:
FechaIni=dt.date(2025, 7, 1)
FechaFin=dt.date(2025, 7, 15)

df_Rec= apiXM.request_data(pydataxm.ReadDB(),    #Se indica el objeto que contiene el serivicio
                        "ListadoRecursos",                #Se indica el nombre de la métrica tal como se llama en el campo metricID
                        "Sistema",                 #Campo que indica si es Sistema, Agente, Recurso, Comercializador
                        FechaIni,       #Corresponde a la fecha inicial de la consulta
                        FechaFin)      #Corresponde a la fecha final de la consulta
# df_Rec=df_Rec.drop('Date',axis=1)

In [18]:
df_Rec1=df_Rec[['Values_Name','Values_Type','Values_Disp','Values_OperStartdate']]
df_Rec1=df_Rec1[(df_Rec1['Values_Type'].isin(['SOLAR','EOLICA']))]
df_Rec1.to_csv('OperacionRenovable.csv')
df_Rec1 = df_Rec1[~df_Rec1['Values_Name'].str.contains('FUTURA', case=False, na=False)]
df_Rec1["nombre_normalizado"] = df_Rec1["Values_Name"].apply(normalizar_nombre)
df_Rec1


,Values_Name,Values_Type,Values_Disp,Values_OperStartdate,nombre_normalizado
26,AUTOG CELSIA SOLAR YUMBO,SOLAR,NO DESPACHADO CENTRALMENTE,2017-09-03,celsia yumbo
42,CELSIA SOLAR BOLIVAR,SOLAR,NO DESPACHADO CENTRALMENTE,2019-02-03,celsia bolivar
48,AGPE NAFERTEX,SOLAR,NO DESPACHADO CENTRALMENTE,2019-11-27,nafertex
50,AGPE TECNOEMPAQUES DE OCCIDENTE,SOLAR,NO DESPACHADO CENTRALMENTE,2019-12-19,tecnoempaques occidente
52,AGPE SFV MCDONALDS SOLEDAD,SOLAR,NO DESPACHADO CENTRALMENTE,2020-02-27,sfv mcdonalds soledad
...,...,...,...,...,...
982,GUAYEPO,SOLAR,DESPACHADO CENTRALMENTE,2024-11-30,guayepo
1000,LA MATA,SOLAR,DESPACHADO CENTRALMENTE,2024-06-27,mata
1085,PARQUE SOLAR TEPUY,SOLAR,DESPACHADO CENTRALMENTE,2024-06-12,tepuy
1086,TRINA-VATIA BSLII,SOLAR,NO DESPACHADO CENTRALMENTE,2021-01-22,trinavatia bslii


In [ ]:
# Ejecutar el fuzzy join para las plantas que están en operación
df_Opera1 = fuzzy_left_join(df_Rec1, "Values_Name", df_proyectos, "Proyecto", umbral=95)[['Values_Name','Values_Type','Values_Disp','Values_OperStartdate','FPO_Ini']]
# df_Opera1 = fuzzy_left_join(df_Rec1, "Values_Name", df_proyectos, "Proyecto", umbral=95)

df_Opera2 = fuzzy_left_join(df_Rec1, "Values_Name", df_data_radar, "PROYECTO", umbral=95)[['Values_Name','Values_Type','Values_Disp','Values_OperStartdate','FPO_Ini']]
# df_Opera2 = fuzzy_left_join(df_Rec1, "Values_Name", df_data_radar, "PROYECTO", umbral=95)
# df_Opera2 = df_Opera2.rename(columns={'FPO': 'FPO_Ini'})

df_Opera=pd.concat([df_Opera1,df_Opera2],axis=0)

df_Opera2.to_csv('RecursosOperacion.csv')


In [21]:
df_Opera

,Values_Name,Values_Type,Values_Disp,Values_OperStartdate,FPO_Ini
0,BOSQUES SOLARES DE LOS LLANOS 5,SOLAR,NO DESPACHADO CENTRALMENTE,2022-09-12,2025-12-31
1,BOSQUES SOLARES DE LOS LLANOS 4,SOLAR,NO DESPACHADO CENTRALMENTE,2022-09-12,2025-12-31
2,FUNDACION,SOLAR,DESPACHADO CENTRALMENTE,2024-06-24,2024-06-24
3,SHANGRI LA,SOLAR,DESPACHADO CENTRALMENTE,2025-02-28,2025-12-31
4,EL PASO,SOLAR,DESPACHADO CENTRALMENTE,2024-03-23,2023-11-30
...,...,...,...,...,...
154,PARQUE SOLAR TEPUY,SOLAR,DESPACHADO CENTRALMENTE,2024-06-12,2023-07-01 00:00:00
155,PARQUE SOLAR TEPUY,SOLAR,DESPACHADO CENTRALMENTE,2024-06-12,2023-10-01 00:00:00
156,PARQUE SOLAR TEPUY,SOLAR,DESPACHADO CENTRALMENTE,2024-06-12,NaN
157,PARQUE SOLAR TEPUY,SOLAR,DESPACHADO CENTRALMENTE,2024-06-12,Diciembre de 2023


In [35]:
# Ruta del archivo Excel
ruta_excel = r'C:\Alejo\PreciosBolsaEnersinc\Informacion_20250716\ProyectosGen.xlsx'
df_hoja = pd.read_excel(ruta_excel, sheet_name='Export')
df_hoja=df_hoja[(df_hoja['Tipo'].isin(['Solar','Eólico']))]
df_hoja["nombre_normalizado"] = df_hoja["Proyecto"].apply(normalizar_nombre)

In [15]:
df_hoja

,Código,Proyecto,CEN [MW],FPO,Tipo,OEF,Área operativa,Tipo OEF,Fecha obligación,Subárea operativa,Promotor,Puntos de Conexión,Fecha de Puesta en Operación Oficial,nombre_normalizado
0,PROG00107,Prosperidad,19.50,2025-12-20,Solar,NO,Caribe,NaN,NaN,Atlantico,LA PROSPERIDAD SOLAR S.A.S.,Salamina 34.5 kV,12/20/2025 00:00:00,prosperidad
2,PROG00356,Windpeshi,200.00,2026-09-30,Eólico,NO,Caribe,Subasta CxC,NaN,GCM,ENEL COLOMBIA SA ESP,Cuestecitas 220 kV,09/30/2026 00:00:00,windpeshi
4,PROG00364,"Atlántico Solar II Polo Nuevo de 9,9 MW",9.90,2025-12-30,Solar,NO,Caribe,NaN,NaN,Atlantico,TECHNOELITE GREEN ENERGY S.A.S. E.S.P,Baranoa 13.8 kV,12/30/2023 00:00:00,atlantico ii polo nuevo 99 mw
6,PROG00396,JK3,45.00,2026-12-31,Eólico,NO,Caribe,NaN,NaN,GCM,AES COLOMBIA & CIA. S.C.A. E.S.P.,Colectora 500 kV,12/01/2026 00:00:00,jk3
7,PROG00397,Parque Eólico Tumawind,200.00,2026-12-31,Eólico,NO,Caribe,Subasta CxC,NaN,GCM,ENEL COLOMBIA SA ESP,Colectora 1 500 kV,12/31/2026 00:00:00,tumawind
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,PROG15810,GD Polaris 2,0.99,2025-10-08,Solar,NO,Suroccidental,NaN,NaN,Cauca-Narino,EVOLTI COMPANY SAS BIC,Remolinos 13.8 kV,NaN,gd polaris 2
402,PROG15836,GD EDQ Solar La Tebaida,0.12,2025-08-15,Solar,NO,Suroccidental,NaN,NaN,CQR,EMPRESA DE ENERGIA DEL QUINDIO S.A. E.S.P.,NaN,NaN,gd edq tebaida
403,PROG15863,GD ERCO Camilocé,0.99,2025-09-08,Solar,NO,Antioquía - Chocó,NaN,NaN,Antioquia,ERCO GENERACION S.A.S. ESP,Amagá 13.2 kV,NaN,gd erco camiloce
404,PROG15866,GD P15-4,0.99,2025-09-22,Solar,NO,Nordeste,NaN,NaN,Santander,ES INVEST COLOMBIA S.A.S. ESP,Puente Sogamoso 34.5/13.2kV,NaN,gd p154


In [47]:
# Ejecutar el fuzzy join para las plantas que están en proyecto
# df_Pro1 = fuzzy_left_join(df_hoja, "Proyecto", df_proyectos, "Proyecto", umbral=95)[['Proyecto_izq','CEN [MW]','FPO','Tipo_izq','OEF_izq','FPO_Ini']]
df_Pro1 = fuzzy_left_join(df_hoja, "Proyecto", df_proyectos, "Proyecto", umbral=90)
df_Pro1 = df_Pro1.rename(columns={'Proyecto_izq': 'Proyecto','Tipo_izq': 'Tipo','OEF_izq': 'OEF'})

df_Pro2 = fuzzy_left_join(df_hoja, "Proyecto", df_data_radar, "PROYECTO", umbral=95)[['Proyecto','CEN [MW]','FPO','Tipo','OEF','FPO_Ini']]


df_Pro=pd.concat([df_Pro1,df_Pro2],axis=0)

df_Pro1.to_csv('RecursosProyectos.csv')

In [38]:
df_hoja1=df_hoja.copy()
# Lista de referencia (derecha)
nombres_ref = df_proyectos["nombre_normalizado"].tolist()

# Buscar mejor match para cada fila del DF izquierdo
matches = df_hoja1["nombre_normalizado"].apply(
    lambda x: process.extractOne(x, nombres_ref) if x else (None, 0)
    )

df_hoja1['Resultado']=matches

df_hoja1=df_hoja1[['Proyecto','nombre_normalizado','Resultado']]

In [39]:
df_hoja1

,Proyecto,nombre_normalizado,Resultado
0,Prosperidad,prosperidad,"(puerta oro, 54)"
2,Windpeshi,windpeshi,"(windpeshi, 100)"
4,"Atlántico Solar II Polo Nuevo de 9,9 MW",atlantico ii polo nuevo 99 mw,"(atlantico, 90)"
6,JK3,jk3,"(jk2, 67)"
7,Parque Eólico Tumawind,tumawind,"(kuisa tumawind, 90)"
...,...,...,...
401,GD Polaris 2,gd polaris 2,"(acacia 2, 86)"
402,GD EDQ Solar La Tebaida,gd edq tebaida,"(orquidea, 49)"
403,GD ERCO Camilocé,gd erco camiloce,"(camelias, 57)"
404,GD P15-4,gd p154,"(melgar 180 mw, 39)"


In [45]:
from fuzzywuzzy import fuzz, process
process.extractOne('atlantico ii polo nuevo 99 mw',nombres_ref,scorer=fuzz.ratio)

c:\Users\eramirez\AppData\Local\Programs\Python\Python311\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


('atlantico', 47)